In [1]:
import pandas as pd
import sqlite3

csv_file_path = r"https://raw.github.com/reuvenkazorer27/sql---Superstore-/main/Sample%20-%20Superstore.csv"
df = pd.read_csv(csv_file_path, encoding='ISO-8859-1')
conn = sqlite3.connect(':memory:')
df.to_sql('superstore', conn, index=False, if_exists='replace')


9994

In [2]:
top_categories_query = """
SELECT 
    Category, 
    SUM(Profit) AS total_profit,
    ROUND(SUM(Profit) / SUM(Sales) * 100, 1) || '%' AS profit_percentage
FROM 
    superstore
GROUP BY 
    Category

UNION ALL

SELECT 
    'Total' AS Category,  
    SUM(Profit) AS total_profit,
    ROUND(SUM(Profit) / SUM(Sales) * 100, 1) || '%' AS profit_percentage
FROM 
    superstore;
"""

top_categories = pd.read_sql(top_categories_query, conn)
print("Top Categories by Total Profit:")
print(top_categories)

for index, row in top_categories.iterrows():
    category = row['Category']
    
    # Query for top subcategories
    subcategory_query = f"""
    SELECT 
        [Sub-Category], 
        SUM(Profit) AS total_profit,
        ROUND(SUM(Profit) / SUM(Sales) * 100, 1) || '%' AS profit_percentage
    FROM 
        superstore
    WHERE 
        Category = '{category}'
    GROUP BY 
        [Sub-Category]

    UNION ALL

    SELECT 
        'Total' AS [Sub-Category],  
        SUM(Profit) AS total_profit,
        ROUND(SUM(Profit) / SUM(Sales) * 100, 1) || '%' AS profit_percentage
    FROM 
        superstore
    WHERE 
        Category = '{category}'
    ORDER BY
        total_profit DESC;
    """
    
    top_subcategories = pd.read_sql(subcategory_query, conn)
    total_row = top_subcategories[top_subcategories['Sub-Category'] == 'Total']
    print(f"\nTotal Profit for {category}:")
    print(total_row)

    top_subcategories = top_subcategories[top_subcategories['Sub-Category'] != 'Total']
    print(f"\nTop Subcategories for {category}:")
    print(top_subcategories)

Top Categories by Total Profit:
          Category  total_profit profit_percentage
0        Furniture    18451.2728              2.5%
1  Office Supplies   122490.8008             17.0%
2       Technology   145454.9481             17.4%
3            Total   286397.0217             12.5%

Total Profit for Furniture:
  Sub-Category  total_profit profit_percentage
1        Total    18451.2728              2.5%

Top Subcategories for Furniture:
  Sub-Category  total_profit profit_percentage
0       Chairs    26590.1663              8.1%
2  Furnishings    13059.1436             14.2%
3    Bookcases    -3472.5560             -3.0%
4       Tables   -17725.4811             -8.6%

Total Profit for Office Supplies:
  Sub-Category  total_profit profit_percentage
0        Total   122490.8008             17.0%

Top Subcategories for Office Supplies:
  Sub-Category  total_profit profit_percentage
1        Paper    34053.5693             43.4%
2      Binders    30221.7633             14.9%
3      Stor

In [3]:
top_products_query = """
SELECT 
    [Product Name], 
    SUM(Sales) AS total_sales,
    SUM(Profit) AS total_profit,
    ROUND(SUM(Profit) / SUM(Sales) * 100, 2) || '%' AS profit_percentage
FROM 
    superstore
GROUP BY 
    [Product Name]
ORDER BY 
    total_sales DESC limit 10;
"""
top_products = pd.read_sql(top_products_query, conn)
print("Products with the Most Sales, Profit, and Profit Percentage:")
print(top_products)

Products with the Most Sales, Profit, and Profit Percentage:
                                        Product Name  total_sales  \
0              Canon imageCLASS 2200 Advanced Copier    61599.824   
1  Fellowes PB500 Electric Punch Plastic Comb Bin...    27453.384   
2  Cisco TelePresence System EX90 Videoconferenci...    22638.480   
3       HON 5400 Series Task Chairs for Big and Tall    21870.576   
4         GBC DocuBind TL300 Electric Binding System    19823.479   
5   GBC Ibimaster 500 Manual ProClick Binding System    19024.500   
6               Hewlett Packard LaserJet 3310 Copier    18839.686   
7  HP Designjet T520 Inkjet Large Format Printer ...    18374.895   
8          GBC DocuBind P400 Electric Binding System    17965.068   
9        High Speed Automatic Electric Letter Opener    17030.312   

   total_profit profit_percentage  
0  2.519993e+04            40.91%  
1  7.753039e+03            28.24%  
2 -1.811078e+03             -8.0%  
3  5.684342e-14              0.0%  

In [4]:
top_10_cities = """SELECT City,SUM(Profit) as total_profit,ROUND(SUM(Profit) / SUM(Sales) * 100, 1) || '%' AS profit_percentange
FROM superstore 
group by city 
ORDER BY total_profit desc limit 10
    ;"""
topc = pd.read_sql(top_10_cities, conn)
print("Cities with the Most Profit, and Profit Percentage:")
print(topc)


Cities with the Most Profit, and Profit Percentage:
            City  total_profit profit_percentange
0  New York City    62036.9837              24.2%
1    Los Angeles    30440.7579              17.3%
2        Seattle    29156.0967              24.4%
3  San Francisco    17507.3854              15.5%
4        Detroit    13181.7908              31.1%
5      Lafayette    10018.3876              40.0%
6        Jackson     7581.6828              30.4%
7        Atlanta     6993.6629              40.7%
8    Minneapolis     6824.5846              40.5%
9      San Diego     6377.1960              13.4%


In [5]:
categories_discount_query = """
-- Average Discount by Category
SELECT 
    Category, 
    ROUND(AVG(Discount) * 100, 2) || '%' AS avg_discount
FROM 
    superstore
GROUP BY 
    Category
ORDER BY 
    avg_discount DESC;
"""

top_subcategories_discount_query = """
WITH Subcategory_Discounts AS (
    SELECT 
        [Sub-Category], 
        Category,
        AVG(Discount) AS avg_discount
    FROM 
        superstore
    GROUP BY 
        [Sub-Category], Category
),
Ranked_Subcategories AS (
    SELECT 
        [Sub-Category],
        Category,
        ROUND(avg_discount * 100, 2) || '%' AS avg_discount,
        ROW_NUMBER() OVER (PARTITION BY Category ORDER BY avg_discount DESC) AS rank
    FROM 
        Subcategory_Discounts
)
SELECT 
    Category,
    [Sub-Category],
    avg_discount
FROM 
    Ranked_Subcategories
WHERE 
    rank <= 5
ORDER BY 
    Category, rank;
"""
cities_discount_query = """
SELECT 
    City, 
    ROUND(AVG(Discount) * 100, 2) || '%' AS avg_discount
FROM 
    superstore
GROUP BY 
    City
ORDER BY 
    AVG(Discount) DESC limit 10;
"""
categories_discount = pd.read_sql(categories_discount_query, conn)
top_subcategories_discount = pd.read_sql(top_subcategories_discount_query, conn)
cities_discount = pd.read_sql(cities_discount_query, conn)

print("Average Discount by Categories:")
print(categories_discount)

print("\nTop 5 Subcategories by Average Discount (for each category):")
print(top_subcategories_discount)

print("\nAverage Discount by Cities:")
print(cities_discount)

Average Discount by Categories:
          Category avg_discount
0        Furniture       17.39%
1  Office Supplies       15.73%
2       Technology       13.23%

Top 5 Subcategories by Average Discount (for each category):
           Category Sub-Category avg_discount
0         Furniture       Tables       26.13%
1         Furniture    Bookcases       21.11%
2         Furniture       Chairs       17.02%
3         Furniture  Furnishings       13.83%
4   Office Supplies      Binders       37.23%
5   Office Supplies   Appliances       16.65%
6   Office Supplies    Fasteners         8.2%
7   Office Supplies    Envelopes        8.03%
8   Office Supplies     Supplies        7.68%
9        Technology     Machines       30.61%
10       Technology      Copiers       16.18%
11       Technology       Phones       15.46%
12       Technology  Accessories        7.85%

Average Discount by Cities:
            City avg_discount
0     Romeoville        80.0%
1  Missouri City        80.0%
2      Deer Par

In [6]:
profit_by_segment_query = """
SELECT 
    Segment, 
    SUM(Profit) AS total_profit,
    ROUND(SUM(Profit) / SUM(Sales) * 100, 2) || '%' AS profit_percentage
FROM 
    superstore
GROUP BY 
    Segment
ORDER BY 
    total_profit DESC;
"""

profit_by_segment = pd.read_sql(profit_by_segment_query, conn)

print("Profit and Profit Percentage by Segment:")
print(profit_by_segment)


Profit and Profit Percentage by Segment:
       Segment  total_profit profit_percentage
0     Consumer   134119.2092            11.55%
1    Corporate    91979.1340            13.03%
2  Home Office    60298.6785            14.03%


In [7]:
sales_by_year_query = """
SELECT 
    substr([Order Date], -4, 4) AS Year,
    SUM([Sales]) AS Total_Sales
FROM 
    superstore
WHERE 
    [Order Date] IS NOT NULL
GROUP BY 
    Year
ORDER BY 
    Total_Sales DESC;
"""

result = pd.read_sql(sales_by_year_query, conn)
print("Top Years by Total Sales:")
print(result)


Top Years by Total Sales:
   Year  Total_Sales
0  2017  733215.2552
1  2016  609205.5980
2  2014  484247.4981
3  2015  470532.5090


In [8]:
cursor = conn.cursor()

try:
    cursor.execute("ALTER TABLE superstore ADD COLUMN Order_Date_Formatted DATE")
except sqlite3.OperationalError:
    pass  

cursor.execute("""
    UPDATE superstore
    SET Order_Date_Formatted = 
        substr([Order Date], -4, 4) || '-' || 
        printf('%02d', CAST(substr([Order Date], 1, instr([Order Date], '/') - 1) AS INT)) || '-' ||
        printf('%02d', CAST(substr([Order Date], instr([Order Date], '/') + 1, instr(substr([Order Date], instr([Order Date], '/') + 1), '/') - 1) AS INT))
""")
conn.commit()
top_months_query = """
SELECT 
    CASE 
        WHEN strftime('%m', Order_Date_Formatted) = '01' THEN 'January'
        WHEN strftime('%m', Order_Date_Formatted) = '02' THEN 'February'
        WHEN strftime('%m', Order_Date_Formatted) = '03' THEN 'March'
        WHEN strftime('%m', Order_Date_Formatted) = '04' THEN 'April'
        WHEN strftime('%m', Order_Date_Formatted) = '05' THEN 'May'
        WHEN strftime('%m', Order_Date_Formatted) = '06' THEN 'June'
        WHEN strftime('%m', Order_Date_Formatted) = '07' THEN 'July'
        WHEN strftime('%m', Order_Date_Formatted) = '08' THEN 'August'
        WHEN strftime('%m', Order_Date_Formatted) = '09' THEN 'September'
        WHEN strftime('%m', Order_Date_Formatted) = '10' THEN 'October'
        WHEN strftime('%m', Order_Date_Formatted) = '11' THEN 'November'
        WHEN strftime('%m', Order_Date_Formatted) = '12' THEN 'December'
    END AS Month_Name,  -- Mapping month number to month name
    SUM([Sales]) AS total_sales  -- Sum of sales for that month
FROM 
    superstore
GROUP BY 
    Month_Name  -- Grouping by the month name
ORDER BY 
    total_sales DESC;  -- Ordering by total sales in descending order
"""

top_months = pd.read_sql(top_months_query, conn)
print("Top Months by Total Sales:")
print(top_months)


Top Months by Total Sales:
   Month_Name  total_sales
0    November  352461.0710
1    December  325293.5035
2   September  307649.9457
3       March  205005.4888
4     October  200322.9847
5      August  159044.0630
6         May  155028.8117
7        June  152718.6793
8        July  147238.0970
9       April  137762.1286
10    January   94924.8356
11   February   59751.2514


In [9]:


day_count_query = """
SELECT 
    CASE strftime('%w', Order_Date_Formatted)
        WHEN '0' THEN 'Sunday'
        WHEN '1' THEN 'Monday'
        WHEN '2' THEN 'Tuesday'
        WHEN '3' THEN 'Wednesday'
        WHEN '4' THEN 'Thursday'
        WHEN '5' THEN 'Friday'
        WHEN '6' THEN 'Saturday'
    END AS DayName,
    COUNT(*) AS OrderCount
FROM 
    superstore
WHERE 
    Order_Date_Formatted IS NOT NULL
GROUP BY 
    strftime('%w', Order_Date_Formatted)
ORDER BY 
    OrderCount DESC;
"""

day_count_result = pd.read_sql_query(day_count_query, conn)

print("Order counts by day of the week:")
print(day_count_result)

Order counts by day of the week:
     DayName  OrderCount
0     Monday        1871
1     Friday        1818
2     Sunday        1710
3   Saturday        1655
4   Thursday        1463
5    Tuesday        1106
6  Wednesday         371


In [10]:
date_diff_query = """
SELECT 
    [Ship Mode],
    SUM(
        julianday(
            substr([Ship Date], -4, 4) || '-' || 
            substr([Ship Date], 4, 2) || '-' || 
            substr([Ship Date], 1, 2)
        ) - julianday(
            substr([Order Date], -4, 4) || '-' || 
            substr([Order Date], 4, 2) || '-' || 
            substr([Order Date], 1, 2)
        )
    ) AS Total_Days_Difference
FROM 
    superstore
WHERE 
    [Order Date] IS NOT NULL 
    AND [Ship Date] IS NOT NULL
GROUP BY 
    [Ship Mode]
ORDER BY 
    Total_Days_Difference DESC;

"""

result = pd.read_sql(date_diff_query, conn)
print("Total difference between Order and Ship:")
print(result)

Total difference between Order and Ship:
        Ship Mode  Total_Days_Difference
0     First Class                 1344.0
1    Second Class                  488.0
2        Same Day                    0.0
3  Standard Class                    NaN


In [11]:
average_days_difference_by_ship_mode = """
SELECT 
    [Ship Mode],
    AVG(
        julianday(
            substr([Ship Date], -4, 4) || '-' || 
            substr([Ship Date], 4, 2) || '-' || 
            substr([Ship Date], 1, 2)
        ) - julianday(
            substr([Order Date], -4, 4) || '-' || 
            substr([Order Date], 4, 2) || '-' || 
            substr([Order Date], 1, 2)
        )
    ) AS Average_Days_Difference
FROM 
    superstore
WHERE 
    [Order Date] IS NOT NULL AND [Ship Date] IS NOT NULL
GROUP BY 
    [Ship Mode]
ORDER BY 
    Average_Days_Difference DESC;
"""

result = pd.read_sql(average_days_difference_by_ship_mode, conn)
print("The Average difference between Orders:")
print(result)

The Average difference between Orders:
        Ship Mode  Average_Days_Difference
0    Second Class                61.000000
1     First Class                51.692308
2        Same Day                 0.000000
3  Standard Class                      NaN


In [12]:
query = """
SELECT 
    Price_Count,  
    COUNT(*) AS Products_With_This_Price_Count  
FROM (
    SELECT 
        [Product ID],
        COUNT(DISTINCT [Sales] / [Quantity]) AS Price_Count 
    FROM 
        superstore
    GROUP BY 
        [Product ID]
) AS Price_Info
GROUP BY 
    Price_Count
ORDER BY 
    Price_Count;
"""
result = pd.read_sql(query, conn)
print("Products with more than one price:")
print(result)

Products with more than one price:
   Price_Count  Products_With_This_Price_Count
0            1                             236
1            2                             686
2            3                             564
3            4                             252
4            5                              91
5            6                              20
6            7                               8
7            8                               2
8            9                               2
9           10                               1
